# Measure waveforms

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from qubecalib import QubeCalib
from qubecalib.neopulse import (
    Capture,
    Flushleft,
    RaisedCosFlatTop,
    Rectangle,
    Sequence,
    Series,
)

In [ ]:
%matplotlib inline

In [ ]:
# load the system settings
qc = QubeCalib("./system_settings.json")

In [ ]:
# check the settings
qc.system_config_database.asdict()

In [ ]:
box_list = [
    "A25",
    "A73",
]

boxes = {}
for box_name in box_list:
    box = qc.create_box(box_name, reconnect=False)
    if not all(box.link_status().values()):
        box.relinkup(use_204b=False, background_noise_threshold=400)
    box.reconnect()
    boxes[box_name] = box
    print(box_name, box.link_status())

In [ ]:
for box in boxes.values():
    print(box.dump_box())

## Capture noise

In [ ]:
capture = Capture(duration=1024)

with Sequence() as seq:
    capture.target("RQ16")
    capture.target("RQ52")

qc.add_sequence(seq)

for status, data, config in qc.step_execute(
    repeats=1,
    dsp_demodulation=False,
):
    for target, iqs in data.items():
        iq = iqs[0].squeeze()
        plt.plot(np.real(iq), label="I")
        plt.plot(np.imag(iq), label="Q")
        plt.title(target)
        plt.legend()
        plt.show()

## Capture reflected waveforms

In [ ]:
pulse = Rectangle(duration=1024, amplitude=0.1)
capture = Capture(duration=3 * 1024)

with Sequence() as seq:
    with Flushleft():
        pulse.scaled(0.5).target("RQ16")
        pulse.shifted(np.pi).scaled(3).target("RQ52")
        capture.target("RQ16", "RQ52")

qc.add_sequence(seq)

for status, data, config in qc.step_execute(
    repeats=3000,
    interval=150 * 1024,
):
    for target, iqs in data.items():
        iq = iqs[0].squeeze()
        plt.plot(np.real(iq), label="I")
        plt.plot(np.imag(iq), label="Q")
        plt.title(target)
        plt.legend()
        plt.show()

### Multiplexed

In [ ]:
pulse = RaisedCosFlatTop(duration=5 * 1024, amplitude=0.1, rise_time=1024)
capture = Capture(duration=10 * 1024)

with Sequence() as seq:
    with Flushleft():
        pulse.target("RQ16", "RQ17", "RQ18", "RQ19", "RQ52", "RQ53", "RQ54", "RQ55")
        capture.target("RQ16", "RQ17", "RQ18", "RQ19", "RQ52", "RQ53", "RQ54", "RQ55")

qc.add_sequence(seq)

for status, data, config in qc.step_execute(
    repeats=3000,
    interval=150 * 1024,
):
    for target, iqs in data.items():
        iq = iqs[0].squeeze()
        plt.plot(np.real(iq), label="I")
        plt.plot(np.imag(iq), label="Q")
        plt.title(target)
        plt.legend()
        plt.show()

In [ ]:
pulse = RaisedCosFlatTop(duration=1024, amplitude=0.1, rise_time=128)
capture = Capture(duration=6 * 1024)

with Sequence() as seq:
    with Flushleft():
        with Series():
            pulse.target("RQ16")
            pulse.target("RQ17")
            pulse.target("RQ18")
            pulse.target("RQ19")
        with Series():
            pulse.target("RQ52")
            pulse.target("RQ53")
            pulse.target("RQ54")
            pulse.target("RQ55")
        capture.target("RQ16", "RQ17", "RQ18", "RQ19", "RQ52", "RQ53", "RQ54", "RQ55")

qc.add_sequence(seq)

for status, data, config in qc.step_execute(
    repeats=3000,
    interval=150 * 1024,
):
    for target, iqs in data.items():
        iq = iqs[0].squeeze()
        plt.plot(np.real(iq), label="I")
        plt.plot(np.imag(iq), label="Q")
        plt.title(target)
        plt.legend()
        plt.show()